# Primeras nociones de deep-learning y redes neuronales 
## Ejemplo chatbot

import sys
!{sys.executable} -m pip list #para saber que tengo instalado

import pickle  #https://docs.python.org/es/3/library/pickle.html  # guardar modelos (o objetos) como secuencia de bytes (unos y ceros)

import sys
!{sys.executable} -m pip install tensorflow

- que es en palabras sencillas
 https://www.incentro.com/es-ES/blog/que-es-tensorflow
 
- link de la libreria
 https://www.tensorflow.org/?gclid=Cj0KCQiA4OybBhCzARIsAIcfn9nLmib7_Bs2vXmVsx1nI5y3gF07NhydNa0Tfgf6rxdBTf1zZJiA08kaAt7XEALw_wcB



import sys
!{sys.executable} -m pip install keras

- tensorflow vs keras, que son?
https://nodd3r.com/blog/keras-vs-tensorflow-vs-pytorch-diferencias-clave-entre-frameworks-de-deep-learning

-   que es una api
https://www.sydle.com/es/blog/api-6214f68876950e47761c40e7/
Una API es una especie de puente que conecta diversos tipos de software o aplicaciones y puede crearse en varios lenguajes de programación. Además de un buen desarrollo, una API debe tener una documentación clara y objetiva para poder facilitar su implementación.

-  link libreria
https://keras.io/examples/   ESTO LES SIRVE PARA LOS PROYECTOS !!!!!!!!!


import sys
!{sys.executable} -m pip install nltk

- que es en palabras sencillas 
https://code.tutsplus.com/es/tutorials/introducing-the-natural-language-toolkit-nltk--cms-28620

- link de la libreria
https://www.nltk.org

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('universal_tagset')
nltk.download('spanish_grammars')
nltk.download('tagsets')
nltk.download('stopwords') # faltaaaaa
nltk.download('omw-1.4') #faltaaaa
#nltk.download()

In [30]:
import json

def guardar_json(datos):
    archivo=open("intents.json","w")
    json.dump(datos,archivo,indent=4)

#intents: grupos de conversaciones tipicas para nuestro objetivo
# patterns: posibles interacciones con el usuario

#dic={"intents:[[{"key":["vlores"]}],"dic2"]}


biblioteca={"intents":
            [
                {"tag":"saludos",
                 "patterns":["hola",
                             "buenos dias",
                             "buenas tardes",
                             "buenas noches",
                             "como estas",
                             "hay alguien ahi?",
                             "hey",
                             "saludos",
                             "que tal"                      
                             ],
                 "responses":["hola soy SIC-BOT , en que puedo ayudarte"
                             ],
                 "context":[""]
                 },
                
                {"tag":"despedidas",
                 "patterns":["chao",
                             "adios",
                             "hasta luego",
                             "nos vemos",
                             "bye",
                             "hasta pronto",
                             "hasta la proxima"
                             ],
                 "responses":["hasta luego, tenga un buen dia"
                             ],
                 "context":[""]
                 },
                {"tag":"agradecimientos",
                 "patterns":["gracias",
                             "muchas gracias",
                             "mil gracias",
                             "muy amable",
                             "se lo agradezco",
                             "fue de ayuda",
                             "gracias por la ayuda",
                             "muy agradecido",
                             "gracias por su tiempo",
                             "ty"
                            ],
                 "responses":["de nada",
                              "feliz por ayudarlo",
                              "gracias a usted",
                              "estamos para servirle",
                              "fue un placer"
                             ],
                 "context":[""]
                },
                {"tag":"norespuesta",
                 "patterns":[],
                 "responses":["no se detecto una respuesta"
                             ],
                 "context":[""]                    
                }
            ] 
        }
guardar_json(biblioteca)

In [31]:
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense,Dropout
from tensorflow.keras.optimizers import SGD

    #stemmer=LancasterStemmer() #opcional esp
lemmatizer=WordNetLemmatizer()#La lematización es el proceso de agrupar las diferentes formas flexionadas de una 
                                #palabra para que puedan ser analizadas como un único elemento
    
# ejemplo de lematizacion: https://es.wikipedia.org/wiki/Lematización
# lema -> palabra completa  : categoria
# casa -> casas             : plurales
# dije -> diré,dijeramos    :tiempos verbales


ignore_words=["?","¿","!","¡"]
data_file= open("intents.json").read() #aqui cargo el archivo en formato json
intents = json.loads(data_file) # aqui convierto el archivo json a diccionario
intents

{'intents': [{'tag': 'saludos',
   'patterns': ['hola',
    'buenos dias',
    'buenas tardes',
    'buenas noches',
    'como estas',
    'hay alguien ahi?',
    'hey',
    'saludos',
    'que tal'],
   'responses': ['hola soy SIC-BOT , en que puedo ayudarte'],
   'context': ['']},
  {'tag': 'despedidas',
   'patterns': ['chao',
    'adios',
    'hasta luego',
    'nos vemos',
    'bye',
    'hasta pronto',
    'hasta la proxima'],
   'responses': ['hasta luego, tenga un buen dia'],
   'context': ['']},
  {'tag': 'agradecimientos',
   'patterns': ['gracias',
    'muchas gracias',
    'mil gracias',
    'muy amable',
    'se lo agradezco',
    'fue de ayuda',
    'gracias por la ayuda',
    'muy agradecido',
    'gracias por su tiempo',
    'ty'],
   'responses': ['de nada',
    'feliz por ayudarlo',
    'gracias a usted',
    'estamos para servirle',
    'fue un placer'],
   'context': ['']},
  {'tag': 'norespuesta',
   'patterns': [],
   'responses': ['no se detecto una respuesta'],


Cuando trabajamos con datos de texto, necesitamos realizar varios preprocesos en los datos antes de hacer un machine
learning o un modelo de Deep learning.

Según los requisitos, necesitamos aplicar varias operaciones para preprocesar los datos.

La creación de tokens es lo más básico y lo primero que puedes hacer con los datos de texto. 

La creación de tokens es el proceso de dividir todo el texto en partes pequeñas, como palabras.

Aquí iteramos a través de los patrones y tokenizamos la oración usando la función nltk.word_tokenize ()
y agregamos cada palabra en la lista de palabras. También creamos una lista de clases para nuestras etiquetas.

In [32]:
words=[]
classes=[]
documents=[]

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        
        
        #tokenizar cada palabra
        
        w=nltk.word_tokenize(pattern) #separamos las oraciones palabra por palabra y cuardamos cada palabra como token
        words.extend(w)
        
        #agrego un array de documentos
        documents.append((w,intent["tag"]))
        
        #añadimos clases  a nuestra lista de clases
        if intent["tag"] not in classes:
            classes.append(intent["tag"])
            
print(words)

print("\n ####################################################################################################### \n")

print(documents)

print("\n ####################################################################################################### \n")

print(classes)



#recorderis
#append añade todo en el mismo formato, ejemplo si append([lista]) lo agrega como lista
#extend añade los elementos uno por uno , ejemplo si extend([lista]) agrega cada elemento

['hola', 'buenos', 'dias', 'buenas', 'tardes', 'buenas', 'noches', 'como', 'estas', 'hay', 'alguien', 'ahi', '?', 'hey', 'saludos', 'que', 'tal', 'chao', 'adios', 'hasta', 'luego', 'nos', 'vemos', 'bye', 'hasta', 'pronto', 'hasta', 'la', 'proxima', 'gracias', 'muchas', 'gracias', 'mil', 'gracias', 'muy', 'amable', 'se', 'lo', 'agradezco', 'fue', 'de', 'ayuda', 'gracias', 'por', 'la', 'ayuda', 'muy', 'agradecido', 'gracias', 'por', 'su', 'tiempo', 'ty']

 ####################################################################################################### 

[(['hola'], 'saludos'), (['buenos', 'dias'], 'saludos'), (['buenas', 'tardes'], 'saludos'), (['buenas', 'noches'], 'saludos'), (['como', 'estas'], 'saludos'), (['hay', 'alguien', 'ahi', '?'], 'saludos'), (['hey'], 'saludos'), (['saludos'], 'saludos'), (['que', 'tal'], 'saludos'), (['chao'], 'despedidas'), (['adios'], 'despedidas'), (['hasta', 'luego'], 'despedidas'), (['nos', 'vemos'], 'despedidas'), (['bye'], 'despedidas'), (['has

Ahora lematizaremos cada palabra y eliminaremos las palabras duplicadas de la lista.
Lematizar es el proceso de convertir una palabra en su forma de lema y luego crear un archivo pickle para almacenar
los objetos de Python que usaremos al predecir.

In [33]:
# w.replace(é, e) # pendiente

#base de datos, verdad absoluta de las palabras -> REFERENCIA
words = [lemmatizer.lemmatize(w.lower()) for w in words  if w not in ignore_words ]


#for w in words:
#    if w not in ignore_words:          # lo mismo de arriba escrito de otra manera
#        word=lemmatizer.lemmatize(w.lower())
    
print(words)

pickle.dump(words,open("words.pkl","wb"))
pickle.dump(classes,open("classes.pkl","wb"))
    


['hola', 'buenos', 'dia', 'buenas', 'tardes', 'buenas', 'noches', 'como', 'estas', 'hay', 'alguien', 'ahi', 'hey', 'saludos', 'que', 'tal', 'chao', 'adios', 'hasta', 'luego', 'no', 'vemos', 'bye', 'hasta', 'pronto', 'hasta', 'la', 'proxima', 'gracias', 'muchas', 'gracias', 'mil', 'gracias', 'muy', 'amable', 'se', 'lo', 'agradezco', 'fue', 'de', 'ayuda', 'gracias', 'por', 'la', 'ayuda', 'muy', 'agradecido', 'gracias', 'por', 'su', 'tiempo', 'ty']


### teoria

https://www.nltk.org/book_1ed/ch03.html

   Method               Functionality
- s.find(t)=	         index of first instance of string t inside s (-1 if not found)
- s.rfind(t)=	     index of last instance of string t inside s (-1 if not found)
- s.index(t)=	     like s.find(t) except it raises ValueError if not found
- s.rindex(t)=	     like s.rfind(t) except it raises ValueError if not found
- s.join(text)=	     combine the words of the text into a string using s as the glue
- s.split(t)=	     split s into a list wherever a t is found (whitespace by default)
- s.splitlines()=	 split s into a list of strings, one per line
- s.lower()=	         a lowercased version of the string s
- s.upper()=	         an uppercased version of the string s
- s.title()=	         a titlecased version of the string s
- s.strip()=	         a copy of s without leading or trailing whitespace
- s.replace(t, u)=	 replace instances of t with u inside s

Ahora, crearemos los datos de entrenamiento en los que proporcionaremos la entrada y la salida.
Nuestra entrada será el patrón y la salida será la clase a la que pertenece nuestro patrón de entrada. 
Pero la computadora no entiende el texto, así que convertiremos el texto en números.

In [48]:
# preparacion para la formacion de nuestra red neuronal 


#en esta primera parte de nuestra red neuronal estamos haciendo un clasificador, words en tags

training=[]
output_empty=[0]*len(classes)# creamos una matriz del numero de patterns con valor inicial 0

for doc in documents: #en doc esta la raw_data -> datos sin procesar
    
    #bag of words
    bag=[]
    #lista de tokens
    pattern_words=doc[0]# doc[0] es la lista de palabras
    # lematizacion del token
    pattern_words= [lemmatizer.lemmatize(word.lower()) for word in pattern_words  if word not in ignore_words ]
    # si la palabra coincide introduzco 1, en caso contrario 0
    
    for palabra in words:
        bag.append(1) if palabra in pattern_words else bag.append(0) 
    
    output_row =list(output_empty)
    output_row[classes.index(doc[1])] = 1 #doc en la posicion 1 es el pattern
    
    training.append([bag,output_row])
    #print(output_row)

training=np.array(training)

print(len(training[0][0]))
print(training)

 
         

52
[[list([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([1, 0, 0])]
 [list([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([1, 0, 0])]
 [list([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([1, 0, 0])]
 [list([0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([1, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([1, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

C:\Users\joseb\AppData\Local\Temp\ipykernel_5720\4292130521.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


In [ ]:
#que hace todo este codigo anterior tan horroroso:
# estamos ENCRIPTANDO

convierte palabras en "booleano"(lista de 1 y 0) y les asigna una categoria 

para asignal la categoria busca si esta en la lista de referencia

como son las categorias:
    [1,0,0]=saludos
    [0,1,0]=despedidas
    [0,0,1]=agradecimientos

    
forma pares:    

hola   :[1, 0, 0, 0, 0, 0, ... [1,0,0] -> 10000 : saludo
buenos :[0, 1, 1, 0, 0, 0, ... [1,0,0] -> 01100 : saludo
         
estos pares son la data que vamos a mandar a la IA para poder crear el modelo
 
         
 data=entradas     target=salidas
c1,c2,c3,c4,c5,c6...s1,s2,s3
1  0  0  0  0  0    1 0  0  -> saludo    